In [2]:
import numpy as np
import pandas as pd
from hazm import stopwords_list

In [8]:
data = pd.read_csv('../data/taghche.csv')
data

,date,comment,bookname,rate,bookID,like
0,1395/11/14,اسم کتاب No one writes to the Colonel\nترجمش...,سرهنگ کسی ندارد برایش نامه بنویسد,0.0,3.0,2.0
1,1395/11/14,"طاقچه عزیز،نام کتاب""کسی به سرهنگ نامه نمینویسد...",سرهنگ کسی ندارد برایش نامه بنویسد,5.0,3.0,2.0
2,1394/06/06,بنظرم این اثر مارکز خیلی از صد سال تنهایی که ب...,سرهنگ کسی ندارد برایش نامه بنویسد,5.0,3.0,0.0
3,1393/09/02,به نظر کتاب خوبی میومد اما من از ترجمش خوشم نی...,سرهنگ کسی ندارد برایش نامه بنویسد,2.0,3.0,0.0
4,1393/06/29,کتاب خوبی است,سرهنگ کسی ندارد برایش نامه بنویسد,3.0,3.0,0.0
...,...,...,...,...,...,...
69824,1398/06/13,من چاپیش رو خوندم خیلی هم لذت بردم.به نظرم موض...,رو به باد,5.0,59636.0,3.0
69825,1398/06/16,کاش یه تخفیف میذاشتن . خیلی دوست دارم بخونمش,تحقیر و توهین‌شده‌ها,5.0,59638.0,2.0
69826,1398/05/26,این کتاب داستان درد و رنج کشاورزان بیکارشده‌ی ...,خوشه‌های خشم,5.0,59645.0,9.0
69827,1398/09/29,کتابی فوق العاده زیبا و عالی,موش‌ها و آدم‌ها,5.0,59646.0,0.0


In [4]:
stopwords_list()

['آخرین',
 'آقای',
 'آمد',
 'آمده',
 'آمده_است',
 'آن',
 'آنان',
 'آنجا',
 'آنها',
 'آنچه',
 'آنکه',
 'آورد',
 'آوری',
 'آیا',
 'ابتدا',
 'اثر',
 'اجرا',
 'اخیر',
 'از',
 'است',
 'اش',
 'اغلب',
 'افراد',
 'افرادی',
 'افزود',
 'البته',
 'اما',
 'امر',
 'امکان',
 'اند',
 'او',
 'اول',
 'اولین',
 'اکنون',
 'اگر',
 'ایشان',
 'این',
 'اینجا',
 'اینکه',
 'با',
 'بار',
 'باره',
 'باز',
 'باشد',
 'باشند',
 'باعث',
 'بالا',
 'باید',
 'بخش',
 'بخشی',
 'بدون',
 'بر',
 'برابر',
 'براساس',
 'برای',
 'برخی',
 'برداری',
 'بروز',
 'بزرگ',
 'بسیار',
 'بسیاری',
 'بعد',
 'بعضی',
 'بلکه',
 'بنابراین',
 'بندی',
 'به',
 'بهتر',
 'بهترین',
 'بود',
 'بودن',
 'بودند',
 'بوده',
 'بوده_است',
 'بی',
 'بیان',
 'بیرون',
 'بیش',
 'بیشتر',
 'بیشتری',
 'بین',
 'تا',
 'تاکنون',
 'تبدیل',
 'تحت',
 'ترتیب',
 'تعداد',
 'تعیین',
 'تغییر',
 'تمام',
 'تمامی',
 'تنها',
 'تهیه',
 'تو',
 'جا',
 'جاری',
 'جای',
 'جایی',
 'جدی',
 'جدید',
 'جریان',
 'جز',
 'جمع',
 'جمعی',
 'حال',
 'حالا',
 'حالی',
 'حتی',
 'حد',
 'حداقل',
 'حدود',

In [17]:
import pandas as pd
from collections import Counter
import re

# Load the dataset
df = data

# Load the list of stop words
stop_words = stopwords_list()

# Function to preprocess the comments
def preprocess(text):
    # Remove punctuation and non-alphanumeric characters (Persian specific)
    text = re.sub(r'[^\w\s\u0600-\u06FF]', ' ', text)
    # Split into words
    words = text.split()
    return words

# Initialize a counter for the stop words
stop_words_counter = Counter()
df['comment'].dropna()
# Process each comment
for comment in df['comment']:
    if isinstance(comment, str):
        words = preprocess(comment)
        # Count the stop words in this comment
        stop_words_in_comment = [word for word in words if word in stop_words]
        stop_words_counter.update(stop_words_in_comment)

# Convert the counter to a pandas DataFrame
stop_words_df = pd.DataFrame(stop_words_counter.items(), columns=['Stop Word', 'Frequency'])

# Display the DataFrame
stop_words_df

,Stop Word,Frequency
0,کسی,2334
1,به,43946
2,که,46096
3,چه,3689
4,از,41331
...,...,...
343,خطر,90
344,وگو,6
345,متر,14
346,افزود,13


In [22]:
import pandas as pd
from collections import Counter
import re

data['category'] = data['rate'].apply(lambda x: 'BAD' if x < 3 else('NEUTRAL' if x < 5 else 'GOOD'))
df = data
stop_words = stopwords_list()

# Function to preprocess the comments
def preprocess(text):
    # Remove punctuation and non-alphanumeric characters (Persian specific)
    text = re.sub(r'[^\w\s\u0600-\u06FF]', ' ', text)
    # Split into words
    words = text.split()
    return words

# Initialize a dictionary to store counters for each category
category_counters = {'GOOD': Counter(), 'BAD': Counter(), 'NEUTRAL': Counter()}

# Process each comment
for index, row in df.iterrows():
    comment = row['comment']
    category = row['category']
    if isinstance(comment, str):
        words = preprocess(comment)
        # Count the stop words in this comment
        stop_words_in_comment = [word for word in words if word in stop_words]
        category_counters[category].update(stop_words_in_comment)

# Create a DataFrame to hold the results
all_stop_words = set(stop_words)  # Unique list of stop words

# Initialize the DataFrame with categories as columns
result_df = pd.DataFrame(all_stop_words, columns=['GOOD', 'BAD', 'NEUTRAL'])
result_df = result_df.fillna(0)  # Fill with zeroes

# Fill the DataFrame with the counts
for category, counter in category_counters.items():
    for word, freq in counter.items():
        result_df.at[word, category] = freq

# Reset index to have 'Stop Word' as a column
result_df = result_df.reset_index().rename(columns={'index': 'Stop Word'})
result_df

ValueError: Shape of passed values is (389, 1), indices imply (389, 3)